In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import textblob
import pandas as pd

from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
!unzip '/content/drive/My Drive/shopee-titles/shopee-product-title-translation-open.zip' 

Archive:  /content/drive/My Drive/shopee-titles/shopee-product-title-translation-open.zip
  inflating: dev_en.csv              
  inflating: dev_tcn.csv             
  inflating: test_tcn.csv            
  inflating: train_en.csv            
  inflating: train_tcn.csv           


In [ ]:
%%bash
pip install jieba
pip install fasttext
pip install hanziconv
pip install fairseq
pip install sacrebleu
pip install googletrans

cd /usr/local/lib/python3.6/dist-packages/fairseq/modules/lightconv_layer
cd ..
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/cuda_utils.cu
cd lightconv_layer
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/lightconv_layer/lightconv_cuda.cpp
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/lightconv_layer/lightconv_cuda.cuh
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/lightconv_layer/lightconv_cuda_kernel.cu
python cuda_function_gen.py
python setup.py install


cd /usr/local/lib/python3.6/dist-packages/fairseq/modules/dynamicconv_layer
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/dynamicconv_layer/dynamicconv_cuda.cpp
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/dynamicconv_layer/dynamicconv_cuda.cuh
wget -L https://raw.githubusercontent.com/pytorch/fairseq/master/fairseq/modules/dynamicconv_layer/dynamicconv_cuda_kernel.cu
python cuda_function_gen.py
python setup.py install
pip install sacremoses subword_nmt


  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015251 sha256=1b9250be081f59c11edc889a199c33713cd308fcfe22897c24e376ac7716de83
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23215 sha256=1041d98df63ca924e8760979deaf70fc4e74b4882e826a79c897821fcb745518
  Stored in directory: /root/.cache/pip/wheels/03/d8/3c/c39898fa9c9ce6e34b0ab4c6604892462d440c743715c94054
Successfully built hanziconv
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2021665 sha256=d95779ce81617df3b17d4f402b11f9407a73a450bef02280fa163ffaf0a0af77
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=6b0dd54

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
dfs = {}

for fn in ['train_en', 'train_tcn']:
    lines = []
    cats = []
    with open(f'{fn}.csv', 'r') as f:
        for line in f: 
            break
        for line in f:
            if line.count(',') == 0:
                continue
            phrases = line.strip().split(',')
            cats.append(phrases[-1])
            line = ','.join(phrases[:-1])
            lines.append(','.join(phrases[:-1]))

    # with open(f'{fn}_cleaned.csv', 'w') as f:
    #     f.writelines(lines)

    dfs[f'{fn}_cleaned'] = pd.DataFrame({'product_title': lines, 'category': cats})
    

files = ['dev_en', 'dev_tcn', 'test_tcn']
for f in files:
    print(f)
    dfs[f] = pd.read_csv(f'{f}.csv', escapechar='\\')

dev_en
dev_tcn
test_tcn


In [ ]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'lightconv.glu.wmt17.zh-en', ... ]

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:305: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/fairseq
creating build/temp.linux-x86_64-3.6/fairseq/clib
creating build/temp.linux-x86_64-3.6/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.6/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=libbleu -D_GLIBCXX_USE_CXX11_ABI=0
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c fairseq/clib/libbleu/module.cpp -o build/temp.linux-x86_64-3.6/fairseq/clib/libbleu/modul

['bart.base',
 'bart.large',
 'bart.large.cnn',
 'bart.large.mnli',
 'bart.large.xsum',
 'bpe',
 'camembert',
 'camembert-base',
 'camembert-base-ccnet',
 'camembert-base-ccnet-4gb',
 'camembert-base-oscar-4gb',
 'camembert-base-wikipedia-4gb',
 'camembert-large',
 'camembert.v0',
 'conv.stories',
 'conv.stories.pretrained',
 'conv.wmt14.en-de',
 'conv.wmt14.en-fr',
 'conv.wmt17.en-de',
 'data.stories',
 'dynamicconv.glu.wmt14.en-fr',
 'dynamicconv.glu.wmt16.en-de',
 'dynamicconv.glu.wmt17.en-de',
 'dynamicconv.glu.wmt17.zh-en',
 'dynamicconv.no_glu.iwslt14.de-en',
 'dynamicconv.no_glu.wmt16.en-de',
 'lightconv.glu.wmt14.en-fr',
 'lightconv.glu.wmt16.en-de',
 'lightconv.glu.wmt17.en-de',
 'lightconv.glu.wmt17.zh-en',
 'lightconv.no_glu.iwslt14.de-en',
 'lightconv.no_glu.wmt16.en-de',
 'roberta.base',
 'roberta.large',
 'roberta.large.mnli',
 'roberta.large.wsc',
 'tokenizer',
 'transformer.wmt14.en-fr',
 'transformer.wmt16.en-de',
 'transformer.wmt18.en-de',
 'transformer.wmt19.de-en',

In [ ]:
dfs['dev_en']

,translation_output
0,Oppo A75 A75S A73 Phone Case Soft Rabbit Silic...
1,SOFT 99 Coating Car Wax Strong Water Watt
2,Low Sugar Mango Dry 250g Be The Royal
3,* the culture Japan Imported Round Top Space C...
4,Hello Kitty Sandals Shoes White/Red Children n...
...,...
995,Hippored Torn Fun Unique Style Straight Jeans ...
996,Kids Set Table Bay - Thin Long Sleeve Home Sui...
997,LONGCHAMP Le Pliage Neo High Density Nylon Bac...
998,IFairies Opening Adjustable Ring ifairies [564...


In [ ]:
with open('dev_zh_cleaned.txt', 'w') as f:
    for line in dfs['dev_tcn'].text.values:
        f.write(process_tcn(line) + '\n')

with open('dev_en_cleaned.txt', 'w') as f:
    for line in dfs['dev_en'].translation_output.values:
        f.write(line + '\n')


In [ ]:
import fairseq
import jieba
from hanziconv import HanziConv
import sacrebleu

# https://github.com/pytorch/fairseq/tree/master/examples/pay_less_attention_paper

zh2en = torch.hub.load('pytorch/fairseq', 'lightconv.glu.wmt17.zh-en', tokenizer='moses', bpe='subword_nmt')

zh2en.cuda()

# The underlying model is available under the *models* attribute
assert isinstance(zh2en.models[0], fairseq.models.lightconv.LightConvModel)


# Translate a sentence
zh2en.translate('你好 世界')
# 'Hello World'
# zh2endyn = torch.hub.load('pytorch/fairseq', 'dynamicconv.glu.wmt17.zh-en', tokenizer='moses', bpe='subword_nmt')


Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 2998309912/2998309912 [03:53<00:00, 12821119.26B/s]


No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'
No module named 'lightconv_cuda'


'Hello World'

In [ ]:
zh2endyn = torch.hub.load('pytorch/fairseq', 'dynamicconv.glu.wmt17.zh-en', tokenizer='moses', bpe='subword_nmt')

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 2994863428/2994863428 [03:52<00:00, 12873246.09B/s]


No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'
No module named 'dynamicconv_cuda'


In [ ]:
zh2endyn.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): LightConvModel(
      (encoder): LightConvEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(48216, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): LightConvEncoderLayer(
            dropout=0.25, relu_dropout=0.0, input_dropout=0.1, normalize_before=False
            (linear1): Linear(in_features=1024, out_features=2048, bias=True)
            (act): GLU(dim=-1)
            (conv): DynamicConv1dTBC(
              1024, kernel_size=3, padding_l=1, num_heads=16, weight_softmax=True, conv_bias=False, renorm_padding=False, in_proj=False, weight_dropout=0.25
              (weight_dropout_module): FairseqDropout()
              (weight_linear): Linear(in_features=1024, out_features=48, bias=False)
            )
            (linear2): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout_module): FairseqD

In [ ]:
def process_tcn(line):
    zh_line = HanziConv.toSimplified(str(line))
    return ' '.join([w for w in jieba.cut(zh_line, cut_all=False) if w.strip() != ''])


def read_txt_to_list(fn):
    res = []
    with open(fn, 'r') as f:
        for line in f:
            res.append(line.strip())
    return res

def write_list_to_txt(lines, fn):
    with open(fn, 'w') as f:
        for line in lines:
            f.write(line + '\n')


zh_file = 'drive/My Drive/shopee-titles/train_zh.txt'

if not os.path.isfile(zh_file):
    zh_lines = [process_tcn(line) for line in dfs['train_tcn_cleaned'].product_title.values]
    write_list_to_txt(zh_lines, zh_file)
else:
    zh_lines = read_txt_to_list(zh_file)

CHUNKSIZE = 50000
n_chunks = len(zh_lines)//CHUNKSIZE + 1
print(f'Number of chunks = {n_chunks}')

zh_chunk_files = [f'zh_chunk{i}.txt' for i in range(n_chunks)]
translate_chunk_files = [f'drive/My Drive/shopee-titles/translated_lightconv_zh_chunk{i}.txt' for i in range(n_chunks)]

for i in range(n_chunks):
    if not os.path.isfile(zh_chunk_files[i]):
        write_list_to_txt(zh_lines[i*CHUNKSIZE:(i+1)*CHUNKSIZE], zh_chunk_files[i])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.972 seconds.
Prefix dict has been built successfully.


Number of chunks = 11


In [ ]:
dev_zh_lines = [process_tcn(line) for line in dfs['dev_tcn'].text.values]
write_list_to_txt(dev_zh_lines, 'dev_zh_lines.txt')

In [ ]:
def translate_zh2en(zh_file, translate_file, batchsize=256):
    zh_lines = read_txt_to_list(zh_file)
    start_time = time.time()
    zh_translate_lightconv = []
    n_batches = len(zh_lines)//batchsize + 1

    for i in range(n_batches):
        print('iter #{}/{}, elapsed time: {:.2f}s'.format(i, n_batches, time.time()-start_time))
        batch = zh_lines[i*batchsize:(i+1)*batchsize]
        translated_batch = zh2en.translate(batch, beam = 8)
        zh_translate_lightconv.extend(translated_batch)

    write_list_to_txt(zh_translate_lightconv, translate_file)

ZH_CHUNKS_TO_TRANSLATE = [6]

for i in ZH_CHUNKS_TO_TRANSLATE:
    zh_cf, trl_cf = zh_chunk_files[i], translate_chunk_files[i]
    if not os.path.isfile(trl_cf):
        print('Translate ' + zh_cf)
        translate_zh2en(zh_cf, trl_cf)

Translate zh_chunk6.txt
iter #0/196, elapsed time: 0.00s
iter #1/196, elapsed time: 7.51s
iter #2/196, elapsed time: 14.89s
iter #3/196, elapsed time: 27.09s
iter #4/196, elapsed time: 35.26s
iter #5/196, elapsed time: 44.94s
iter #6/196, elapsed time: 51.96s
iter #7/196, elapsed time: 61.54s
iter #8/196, elapsed time: 70.03s
iter #9/196, elapsed time: 82.56s
iter #10/196, elapsed time: 92.41s
iter #11/196, elapsed time: 101.65s
iter #12/196, elapsed time: 109.96s
iter #13/196, elapsed time: 119.16s
iter #14/196, elapsed time: 127.02s
iter #15/196, elapsed time: 138.62s
iter #16/196, elapsed time: 146.34s
iter #17/196, elapsed time: 156.08s
iter #18/196, elapsed time: 165.95s
iter #19/196, elapsed time: 174.06s
iter #20/196, elapsed time: 181.40s
iter #21/196, elapsed time: 190.81s
iter #22/196, elapsed time: 198.39s
iter #23/196, elapsed time: 207.68s
iter #24/196, elapsed time: 217.12s
iter #25/196, elapsed time: 227.86s
iter #26/196, elapsed time: 237.06s
iter #27/196, elapsed time:

In [ ]:
translate_zh2en('dev_zh_lines.txt', 'drive/My Drive/shopee-titles/translated_lightconv_dev_zh.txt')

iter #0/4, elapsed time: 0.00s
iter #1/4, elapsed time: 5.96s
iter #2/4, elapsed time: 11.54s
iter #3/4, elapsed time: 17.21s


In [ ]:
!head -n 10 train_zh.txt

In [ ]:
!head -n 10 drive/My\ Drive/shopee-titles/zh_translate_lightconv.txt

In [ ]:
# # zh2endyn.translate('小拇指 鞋坊 adidas superstar 80s dlx 金标 贝壳 头 白红 时尚 男女 鞋 adidas 板鞋 时尚 情侣 鞋 adids 男鞋 女鞋 36 - 44', beam=8)
dev_tcn = dfs['dev_tcn'].text.values
dev_zh = [process_tcn(a) for a in dev_tcn]
dev_en = dfs['dev_en'].translation_output.values
trans1 = zh2en.translate(dev_zh, beam=16)
trans2 = zh2endyn.translate(dev_zh, beam=16)

In [ ]:
trans [t1 if len(tcn)>30 else t2 for t1, t2, tcn in zip()]

In [ ]:
import re
from typing import List

import regex
from sacrebleu import corpus_bleu

OTHERS_PATTERN = regex.compile(r'\p{So}')


def shopee_eval(preds: List[str], refs: List[str]) -> float:
    """BLEU score computation.

    Strips all characters belonging to the unicode category "So".
    Tokenize with standard WMT "13a" tokenizer.
    Compute 4-BLEU.

    Args:
        preds (List[str]): List of translated texts.
        refs (List[str]): List of target reference texts.
    """
    preds = [OTHERS_PATTERN.sub(' ', text) for text in preds]
    refs = [OTHERS_PATTERN.sub(' ', text) for text in refs]
    return corpus_bleu(
        preds, [refs],
        lowercase=True,
        tokenize='13a',
        use_effective_order= False
    ).score

#  ('IFairies Shoulder Messenger Bag Shoulder Bag 【 49146 】',
def post_process(line):
    line = line.replace('-', ' - ')
    line = line.replace('[', ' 】')
    line = line.replace('[', '【 ')
    line = line.replace('  ', ' ')
    line = line.replace('  ', ' ')
    return line

for beam in range(1,16):
    trans1 = [post_process(line) for line in zh2en.translate(dev_zh, beam=beam)]
    print(beam, shopee_eval(dev_en, trans1))

1 29.61639208717485
2 29.90893936681329
3 30.561801970869652
4 30.871153786067573
5 30.99717241050713
6 31.349587918719866
7 31.366437848168182
8 31.34761450366679
9 31.334267626666584
10 31.521912412746744
11 31.493725861556264
12 31.47342216904188
13 31.465449992038828
14 31.403130167715947
15 31.408236494898258


In [ ]:
for beam in range(1,16):
    trans1 = [post_process(line) for line in zh2endyn.translate(dev_zh, beam=beam)]
    print(beam, shopee_eval(dev_en, trans1))

1 28.41365535220877
2 29.69533191367675
3 29.858194696472943
4 30.255306163129614
5 30.4812345472558
6 30.369630813468557
7 30.518815229136365
8 30.836793572675255
9 30.660489372176713
10 30.521656301302244
11 30.59820362772005
12 30.574116675075413
13 30.493524479885057
14 30.418824329222947
15 30.37693679078742


In [ ]:
test_zh = [process_tcn(line) for line in dfs['test_tcn'].text.values]

In [ ]:
write_list_to_txt(test_zh, 'test_zh.txt')
translate_zh2en('test_zh.txt', 'test_translated.txt')

iter #0/40, elapsed time: 0.00s
iter #1/40, elapsed time: 6.09s
iter #2/40, elapsed time: 11.80s
iter #3/40, elapsed time: 17.52s
iter #4/40, elapsed time: 23.13s
iter #5/40, elapsed time: 28.73s
iter #6/40, elapsed time: 35.31s
iter #7/40, elapsed time: 41.14s
iter #8/40, elapsed time: 46.84s
iter #9/40, elapsed time: 52.36s
iter #10/40, elapsed time: 57.99s
iter #11/40, elapsed time: 63.74s
iter #12/40, elapsed time: 71.82s
iter #13/40, elapsed time: 77.48s
iter #14/40, elapsed time: 83.06s
iter #15/40, elapsed time: 88.41s
iter #16/40, elapsed time: 94.36s
iter #17/40, elapsed time: 102.38s
iter #18/40, elapsed time: 107.70s
iter #19/40, elapsed time: 113.19s
iter #20/40, elapsed time: 119.07s
iter #21/40, elapsed time: 124.69s
iter #22/40, elapsed time: 130.34s
iter #23/40, elapsed time: 135.87s
iter #24/40, elapsed time: 141.82s
iter #25/40, elapsed time: 147.29s
iter #26/40, elapsed time: 153.18s
iter #27/40, elapsed time: 161.24s
iter #28/40, elapsed time: 167.26s
iter #29/40, e

In [ ]:
test_translations = read_txt_to_list('test_translated.txt')
dev_translations = zh2en.translate(dev_zh, beam=8)

In [ ]:
test_translations_post = []
for zhline, transline in zip(test_zh, test_translations):
    transline = transline.replace('-', ' - ')
    if ']' not in zhline:
        transline = transline.replace(']', ' 】')
    if '[' not in zhline:
        transline = transline.replace('[', '【 ')
    transline = transline.replace('  ', ' ')
    transline = transline.replace('  ', ' ')
    test_translations_post.append(transline)


In [ ]:
dev_translations2 = zh2endyn.translate(dev_zh, beam=8)

In [ ]:
test_translations_cleaned = [OTHERS_PATTERN.sub(' ', text) for text in test_translations]

In [ ]:
shopee_eval(['KAWECO SUpra Series Brass Pen, Germany'], ['KAWECO SUpra Series Brass Pen, Germany'])

100.00000000000004

In [ ]:
test_translations_df = pd.DataFrame({'translation_output': test_translations_post})
test_translations_df.to_csv('submit.csv', index=False)

In [ ]:
# dev_en_cleaned = [OTHERS_PATTERN.sub(' ', text) for text in dev_en]
# dev_translations_cleaned = [OTHERS_PATTERN.sub(' ', text) for text in dev_translations]
# dev_translations2_cleaned = [OTHERS_PATTERN.sub(' ', text) for text in dev_translations2]

In [ ]:
# import time
# import googletrans
# from googletrans import Translator
# googletrans.LANGUAGES

# translator = Translator(service_urls=[
#     'translate.google.com',
#     'translate.google.com.vn'
# ])

# all_translations = []
# for i in range(50):
#     print(i)
#     batch = test_sentences[i*200:(i+1)*200]
#     translations = translator.translate(batch.tolist(), dest='en', src='zh-tw')
#     for t in translations:
#         all_translations.append(t.text)
#     time.sleep(5)

# with open('/content/drive/My Drive/shopee-titles/shopee_googletranslate.csv', 'w') as f:
#     for line in all_translations:
#         f.write(line + '\n')
# with open('shopee_googletranslate.csv', 'w') as f:
#     for line in all_translations:
#         f.write(line + '\n')

In [ ]:
# !pip install mstranslate


In [ ]:
# import os, requests, uuid, json
# # key_var_name = 'TRANSLATOR_TEXT_SUBSCRIPTION_KEY'
# # if not key_var_name in os.environ:
# #     raise Exception('Please set/export the environment variable: {}'.format(key_var_name))
# subscription_key = 'b1295591752c41a49e1f765364ac9282'

# # endpoint_var_name = 'TRANSLATOR_TEXT_ENDPOINT'
# # if not endpoint_var_name in os.environ:
# #     raise Exception('Please set/export the environment variable: {}'.format(endpoint_var_name))
# endpoint = 'https://api.cognitive.microsofttranslator.com/'

# path = '/translate?api-version=3.0'
# params = '&to=en'
# constructed_url = endpoint + path + params
# headers = {
#     'Ocp-Apim-Subscription-Key': subscription_key,
#     'Content-type': 'application/json',
#     'X-ClientTraceId': str(uuid.uuid4())
# }
# body = [{
#     'text': '兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092	!'
# }]
# request = requests.post(constructed_url, headers=headers, json=body)
# response = request.json()

In [ ]:
# titles = dfs['train_tcn_cleaned'].product_title.values
# batches = []
# for i in range(11):
#     batch = titles[i*50000:(i+1)*50000]
#     batches.append(batch)

# for i in range(11):
#     with open(f'train_tcn_cleaned_batch{i}.txt', 'w') as f:
#         for line in batches[i]:
#             f.write(line + '\n')